# 접근 방식

Baseline의 Randomforest 지도학습 결과를 보았을 때 0.997 정도의 매우 높은 성능을 보여 Level7만 잘 구별한다면 좋은 성능을 얻을 수 있을 것이라고 생각했습니다.

7을 구별하기 위해서 비지도 학습으로 접근을 시도했고 오토인코더, one-class-svm, isolated-randomforest 등의 비지도 학습 방법을 적용해 보았으나 기존과 단어 몇개만 다른 경우들이 7에 있어서 인지 성능이 좋지 않았습니다. 그 중에서 오토인코더가 성능이 괜찮았는데 나중에 따로 "렛서펜더"님이 오토인코더 관련 코드를 공유해 주실 예정입니다.

결론적으로 로그 데이터의 전처리를 진행하고 Baseline에서 Validation을 통해 등급별로 threshold를 상세 설정한 점수가 가장 높게 나왔습니다. Validation으로 주어진 Level 7 이외에도 7로 추정되는 문장들을 추가로 threshold로 잡아주었는데 7로 추정한 방식은 아래의 링크에 설명되어 있습니다.

[참고]  
https://dacon.io/competitions/official/235717/codeshare/2536?page=1&dtype=recent (대회 베이스라인)  
https://dacon.io/competitions/official/235717/codeshare/2539?page=1&dtype=recent (10duck 님의 데이터 전처리 및 EDA)  
https://dacon.io/competitions/official/235717/support/403102?page=1&dtype=recent (taegu 님의 threshold 조정 문의 글)

# Data & Libaray IMPORT

In [1]:
import pandas as pd
import numpy as np
import os 
import pandas as pds
from dask import dataframe
import re
import numpy as np
import seaborn as sbn
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import wordnet

In [2]:
train = pd.read_csv("train.csv")
test= pd.read_csv("test.csv")

In [3]:
len(test)

1418916

# EDA
### 1. 다양한 기호 및 숫자 제거

In [4]:
lit = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
def mask(tt):
    tt=tt.apply(lambda x: re.sub(r'(\\n)',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[^a-zA-Zㄱ-ㅣ가-힣0-9:=\s\(\)./,\<\>]+',' ',x))
    #tt=tt.apply(lambda x: re.sub(r' ?(?P<note>[:=\(\)./,\<\>]) ?', ' \g<note> ', x))
    tt=tt.apply(lambda x: re.sub(r'[0-9]+',' ',x))
    tt=tt.apply(lambda x: re.sub(r"':/()",' ',x))
    tt=tt.apply(lambda x: re.sub(r':',' ',x))
    tt=tt.apply(lambda x: re.sub(r',',' ',x))
    # = tt.apply(lambda x: re.sub(r'(',' ',x))
    #t = tt.apply(lambda x: re.sub(r')',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ',x))
    for st in lit:
        st = " "+st + " "
        tt=tt.apply(lambda x: re.sub(st,' ',x))
    tt=tt.apply(lambda x: re.sub(r'\s+',' ',x))
    
    return tt

In [5]:
train['pre_log'] = mask(train.full_log)
test['pre_log'] = mask(test.full_log)

### 2. 영어 단어가 아니거나 3글자 미만인지인 경우에는 삭제
+ 단어가 영어단어가 아니거나 3글자 미만이면 False 반환

In [6]:
#포함되어 있는 단어 모음 생성
def count_word(data):
    tem = list(data['pre_log'].str.split(" "))
    all_word = []
    for word in tem:
        all_word.extend(word)
    words = pd.Series(all_word)
    return words.value_counts()

In [7]:
train_count = count_word(train)
test_count = count_word(test)

In [8]:
train_words = list(set(train_count.index))
test_words = list(set(test_count.index))

In [9]:
#영어 단어 + 3글자 이상인 경우만 True
from nltk.corpus import words
def check_words(word_list: list):
    re = [False] * len(word_list)
    for i,word in enumerate(word_list):
        if len(word) < 3:
            continue
        word = word.lower()
        if word in words.words():
            re[i] = True
    return re

word_list = ['Promiscuous', 'ab', 'nihongo', 'abstract', 'pedo','gid']

In [10]:
#check_words는 단어인지를 확인해주고 결과를 t/f로 반환
check_words(word_list)

[True, False, False, True, False, True]

In [11]:
train_tf = check_words(train_words)
test_tf = check_words(test_words)

In [12]:
#단어인지 확인해주는 딕셔너리 만들어주기 (train_isword,test_isword)
train_isword = dict()
test_isword = dict()

total_words = list(train_words)
test_words = list(test_words)

for i in range(len(train_words)):
    train_isword[train_words[i].lower()] = train_tf[i]
    
for i in range(len(test_words)):
    test_isword[test_words[i].lower()] = test_tf[i]
    

### 3. Train, Test 에 각각 적용
##### 같은 결과이지만 시간 측면에서 아래의 방법을 선택했습니다. (위는 주석 처리)
- 모든 문장에 개별적으로 접근하여 영어가 아닌 단어를 제거하는 코드 (약 3~4일)
- 미리 사용된 단어들을 모아서 딕셔너리를 만들어서 사용한 경우(약 10분)

In [13]:
import copy

def checking_train(data):
    re = [False] * len(data)
    for i,word in enumerate(data):
        if train_isword[word]:
            re[i] = True
    return re


def checking_test(data):
    re = [False] * len(data)
    for i,word in enumerate(data):
        if test_isword[word]:
            re[i] = True
    return re



def cutt_train(data):
    data = data.lower()
    splited = data.split(" ")
    check = checking_train(splited)
    c = np.array(splited)
    real_words = list(c[check])    
    tem = " ".join(real_words)
    #tem = tem.lower() 
    return tem

def cutt_test(data):
    data = data.lower()
    splited = data.split(" ")
    check = checking_test(splited)
    c = np.array(splited)
    real_words = list(c[check])  
    tem = " ".join(real_words)
    #tem = tem.lower()
    return tem

In [14]:
train['cut'] = train['pre_log'].map(cutt_train)
test['cut'] = test['pre_log'].map(cutt_test)

In [15]:
del train['full_log']
del test['full_log']

del train['pre_log']
del test['pre_log']


train.to_csv("sub_train3.csv",index=False)
test.to_csv("sub_test3.csv",index=False)


train.to_csv("only_train.csv",index =False)
test.to_csv("only_test.csv",index =False)

### 4. 나중에 사용될 inornot 리스트

train 데이터에 100퍼센트 동일한 문장이 있는지 확인해주는 변수 입니다. 

In [ ]:
train_bag = train['cut'].unique()
len(train_bag)

In [ ]:
#inornot을 통해서 train안에 같은 문장이 있으면 0 없으면 1
from tqdm import tqdm
inornot = np.zeros(len(test))
for i in tqdm(range(len(test))):
    tem = test.iloc[i]['cut']
    if tem not in train_bag:
        inornot[i] = 1

# 모델

train = pd.read_csv("sub_train3.csv")
test= pd.read_csv("sub_test3.csv")

In [16]:
train.head(20)

,id,level,cut
0,0,0,type error warning collection level error erro...
1,1,0,action with response code type unavailable exc...
2,2,0,type error warning collection level error erro...
3,3,0,type error warning collection level error erro...
4,4,1,type audit arch success yes exit gid suid none...
5,5,1,type audit arch success yes exit gid suid none...
6,6,0,type error warning collection level error erro...
7,7,0,warn resurrect connection dead instance but go...
8,8,0,type error warning collection level error erro...
9,9,0,type log warning message error living share no...


In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

#train_idx = train[train['cut'] == '']['cut'].index
#test_idx = test[test['cut'] == '']['cut'].index

#train['cut'].iloc[train_idx] = np.nan
#test['cut'].iloc[train_idx] = np.nan

#남은 단어가 없는 경우에는 'missing' 으로 대체해주었습니다.
train['cut'] = train['cut'].replace('','missing',regex=True)
test['cut'] = test['cut'].replace('','missing',regex=True)



train = train.fillna("missing")
test = test.fillna("missing")

train

,id,level,cut
0,0,0,type error warning collection level error erro...
1,1,0,action with response code type unavailable exc...
2,2,0,type error warning collection level error erro...
3,3,0,type error warning collection level error erro...
4,4,1,type audit arch success yes exit gid suid none...
...,...,...,...
472967,472967,0,error
472968,472968,1,type audit arch success yes exit gid suid none...
472969,472969,0,type log error task manager message poll for w...
472970,472970,0,type error warning collection level error erro...


In [18]:
train_text=list(train['cut'])
train_level=np.array(train['level'])

test_text=list(test['cut'])

In [19]:
vectorizer=CountVectorizer(analyzer="word", max_features=20000)
train_features=vectorizer.fit_transform(train_text)
test_features=vectorizer.transform(test_text)

In [20]:
forest=RandomForestClassifier(n_estimators=100,random_state = 1 )
forest.fit(train_features,train_level)

RandomForestClassifier(random_state=1)

In [21]:
results=forest.predict(test_features)
results_proba=forest.predict_proba(test_features)

In [22]:
results[np.where((np.max(results_proba, axis=1)<0.5) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.5) & (results == 1))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 2))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.95) & (results == 3))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 4))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 5))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 6))[0]]=7

In [23]:
results[np.where((np.max(results_proba, axis=1)<0.94001) & (np.max(results_proba, axis=1) > 0.93999) & (results == 1))[0]]=7 #929로 바꿔보기
results[np.where((np.max(results_proba, axis=1)<0.611657) & (np.max(results_proba, axis=1) > 0.6116568) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.571657) & (np.max(results_proba, axis=1) > 0.5716568) & (results == 0))[0]]=7 # 
results[np.where((np.max(results_proba, axis=1)<0.68001) & (np.max(results_proba, axis=1) > 0.67999) & (results == 5))[0]]=7 #Prevent


# 결과 저장

In [24]:
submission=pd.read_csv('sample_submission.csv')
submission['level']=results
submission['level'].value_counts()

0    1002369
1     396443
3      12813
5       6418
7        780
4         34
2         34
6         25
Name: level, dtype: int64

In [25]:
submission.to_csv("tem_answer.csv",index = False)

# 추가작업

In [ ]:
submission['inornot'] = pd.Series(inornot)

In [ ]:
cal = test_ans[(test_ans['level']==7) & (test_ans['inornot'] == 0)].index
not7_id = list(test.iloc[cal]['id'])
pd.Series(not7_id).to_csv("not7_id.csv",index= False)

# 추가 작업

위의 방식으로 많이 전처리를 하면 7을 걸러내기 위한 threshold 를 잡기는 유리하지만, 지도 학습 그 자체의 성능은 떨어지는 것을 확인 하였습니다. 따라서, 7를 제외한 나머지 등급에는 전처리가 덜 진행된 Data로 지도학습 분류를 하였습니다. TfidfVectorizer 를 사용했습니다. 

In [26]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [27]:
train['full_log']=train['full_log'].str.replace(r'[0-9]', '<num>')
test['full_log']=test['full_log'].str.replace(r'[0-9]', '<num>')

In [28]:
#train[train['full_log'] == '']['cut'] = np.nan
#test[test['full_log'] == '']['cut'] = np.nan
#train['cut'] = train['cut'].replace('','missing',regex=True)
#test['cut'] = test['cut'].replace('','missing',regex=True)


train = train.fillna("missing")
test = test.fillna("missing")

In [29]:
train_text=list(train['full_log'])
train_level=np.array(train['level'])

test_text=list(test['full_log'])
#ids=list(test['id'])

In [30]:
vectorizer=TfidfVectorizer(analyzer="word", max_features=20000)
train_features=vectorizer.fit_transform(train_text)
test_features=vectorizer.transform(test_text)

In [31]:
forest=RandomForestClassifier(n_estimators=100,random_state = 1 )
forest.fit(train_features,train_level)

RandomForestClassifier(random_state=1)

In [76]:
results=forest.predict(test_features)
results_proba=forest.predict_proba(test_features)

In [77]:
submission=pd.read_csv('sample_submission.csv')
submission['level']=results
submission['level'].value_counts()

0    1002770
1     396532
3      12997
5       6524
4         34
2         34
6         25
Name: level, dtype: int64

In [78]:
not7_id = pd.read_csv('not7_id.csv')
not7_id = list(not7_id['0'])

In [79]:
#위에서 잡은 7 합치기 & train에 있는 경우는 사용하지 않기
only = pd.read_csv("tem_answer.csv")
le7 = list(only[only['level']==7]['id'])
le7_update = list((set(le7))-set(not7_id))
print(len(le7))
print(len(le7_update))
idx = submission[submission['id'].isin(le7_update)].index
submission.iloc[idx] = 7
submission['level'].value_counts()

780
581


0    1002512
1     396348
3      12915
5       6467
7        581
4         34
2         34
6         25
Name: level, dtype: int64

In [80]:
who = submission.level
submission=pd.read_csv('sample_submission.csv')
submission['level'] = who
submission['level'].value_counts()

0    1002512
1     396348
3      12915
5       6467
7        581
4         34
2         34
6         25
Name: level, dtype: int64

In [36]:
submission.to_csv("final_answer.csv",index = False)